# Comparing filtering influence of different reference and sync tags

In [ ]:

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [9]:
require(PBSmapping)
library(ggplot2)
require(grid)
require(gridExtra)
require(sp)
library(IRdisplay)
library(repr)
library(splancs)


In [ ]:
#my_files = paste0("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/Positions/TRANSMITTER-", tests, "-CALC-POSITIONS.csv")
#my_files

In [ ]:
all_pos$LAT

In [67]:
# read data

all_pos <- read.csv("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/Positions/ALL-CALC-POSITIONS.csv", stringsAsFactors=FALSE)
tagll <- data.frame(X = all_pos$LON, Y=all_pos$LAT)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
all_pos$Easting<- xyrec[,1]
all_pos$Northing<- xyrec[,2]

syncRec <- read.csv("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/RefSyncStationLocations.csv")
# known Easting and Northing (UTM) for fixed tag location.
tagll <- data.frame(X = syncRec$Longitude, Y=syncRec$Latitude)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
syncRec$Easting<- xyrec[,1]
syncRec$Northing<- xyrec[,2]

# preparing receiver data for use in plots
rec2016 <- read.csv("~/Documents/DataHam/VPS/VPS-INBO-01-Results20160304/onlyStationLocations.csv")
tagll <- data.frame(X = rec2016$Longitude, Y = rec2016$Latitude)
attr(tagll, "projection") <- "LL"
xyrec <- convUL(tagll) * 1000
rec2016$Easting<- xyrec[,1]
rec2016$Northing<- xyrec[,2]

# actual vps fish data
# eel ID's: 100 to 109, 26514, 38725 to 38740 but 34, 35, 38 and 39 missing
eelIDs <- c(100:109,26514,38725:38733,38736,38737,38740)
pos <- all_pos[all_pos$TRANSMITTER<38741,]
rownames(pos) <- seq(length=nrow(pos))

# chull gives an integer vector giving the indices of the unique points lying on the convex hull, in clockwise order
z <- chull(rec2016$Easting,rec2016$Northing)
# to make a polygon from this: re-index the polygon
recPoly <- rec2016[z,]
rownames(recPoly) <- seq(length=nrow(recPoly))

#Determine of each position whether it lies in or out the polygon
pos$inPoly <- point.in.polygon(pos$Easting, pos$Northing, recPoly$Easting,recPoly$Northing)
#if max = 1: all points lie in or out the polygon, none on the edge
if (max(pos$inPoly) == 1) {    
    print('Check: No points on the edge')}

convUL: For the UTM conversion, automatically detected zone 31.
convUL: Converting coordinates within the northern hemisphere.
convUL: For the UTM conversion, automatically detected zone 31.
convUL: Converting coordinates within the northern hemisphere.
convUL: For the UTM conversion, automatically detected zone 31.
convUL: Converting coordinates within the northern hemisphere.


[1] "Check: No points on the edge"


In [53]:
length(pos[[1]])

[1] 7409

In [68]:
#set variables

# average tag transmission rate of stationary tags (ref and sync) 
# = average random delay (600)+ average transmission length (3.6)
transRateStat <- 603.6
# Accuracy Goals: These are determined by your reserach questions and analysis needs.
# Outlier Maximum (Trevor: 15m)
OutlierGoal <- 15
# Accuracy Goal for all Points: 2DRMS <= accGoal (Trevor: 6m)
accGoal <- 2.5
# Goal for average accuracy if studying path lengths (1 order of magnitude less than max path length) (Trevor: 1.77)
avgAccGoal<- 0.66
tests <- c('R1','R2','R3','S7','S8','S10','S12','S13','S14','S15','S16','S17','S18' )

#evaluation of baseline position confidence
confStats <- data.frame(tests)
confStats$posYield <- NA
confStats$avgAcc <- NA
confStats$medAcc <- NA
confStats$propAcc <- NA
confStats$outlierCount <- NA
confStats$totalPoints <- NA
confStats$nbBins <- NA
confStats$intercept <- NA
confStats$slope <- NA
confStats$rsquared <- NA
confStats$useful <- NA

confStatsIN <- data.frame(tests)
confStatsIN$posYield <- NA
confStatsIN$avgAcc <- NA
confStatsIN$medAcc <- NA
confStatsIN$propAcc <- NA
confStatsIN$outlierCount <- NA
confStatsIN$totalPoints <- NA
#confStatsIN$nbBins <- NA
#confStatsIN$intercept <- NA
#confStatsIN$slope <- NA
#confStatsIN$rsquared <- NA
#confStatsIN$useful <- NA

confStatsOUT <- data.frame(tests)
confStatsOUT$posYield <- NA
confStatsOUT$avgAcc <- NA
confStatsOUT$medAcc <- NA
confStatsOUT$propAcc <- NA
confStatsOUT$outlierCount <- NA
confStatsOUT$totalPoints <- NA
confStatsOUT$nbBins <- NA
confStatsOUT$intercept <- NA
confStatsOUT$slope <- NA
confStatsOUT$rsquared <- NA
confStatsOUT$useful <- NA

#evaluation of HPE-influence

#tests_AllHPE <- array(list(), dim=c(length(tests),1))

tests_AllHPE <- list(tests)

In [84]:

for (t in 1:length(tests)){
TEST <- tests[t]
    estPos1 <- all_pos[all_pos$TRANSMITTER==TEST,]
    
    # tag positions in/out receiver array
    estPos1$inPoly <- point.in.polygon(estPos1$Easting, estPos1$Northing, recPoly$Easting,recPoly$Northing)
    
    # Take true measured position from syncRec
    estPos1East <- subset(syncRec, Station.Name == TEST)$Easting
    estPos1North <-subset(syncRec, Station.Name == TEST)$Northing
    estPos1East <- ifelse(is.na(estPos1East), median(estPos1$Easting),estPos1East)
    estPos1North <- ifelse(is.na(estPos1North), median(estPos1$Northing),estPos1North)
    estPos1$error <- sqrt( (estPos1$Easting-(estPos1East))^2 + (estPos1$Northing-(estPos1North))^2)

    #What is the fix rate based on transmit rate and test length
    #convert date time
    tagDateTime<- as.POSIXct((estPos1$DATETIME), tz='UTC')
    testLength<- as.numeric(max(tagDateTime))-as.numeric(min(tagDateTime))
    #number of transmissions that should occur
    possibleFixes <- (as.numeric(max(tagDateTime))-as.numeric(min(tagDateTime)))/transRateStat
    #total number of transmissions heard
    actualFixes <- length(estPos1$DATETIME)
    #Fix Rate for static test (= position yield)
    fixRateStat <- (actualFixes/possibleFixes)*100
    
    confStats$posYield[t] <- fixRateStat
    confStats$avgAcc[t] <- mean(estPos1$error)
    confStats$medAcc[t] <- median(estPos1$error)
    confStats$propAcc[t] <- length(which( estPos1$error < accGoal ))/length(estPos1$error)
    confStats$outlierCount[t] <- length(which( estPos1$error > OutlierGoal ))
    confStats$totalPoints[t] <- length(estPos1$error)
    
    confStatsIN$posYield[t] <- length(estPos1$error[estPos1$inPoly==1])/possibleFixes
    confStatsIN$avgAcc[t] <- mean(estPos1$error[estPos1$inPoly==1])
    confStatsIN$medAcc[t] <- median(estPos1$error[estPos1$inPoly==1])
    confStatsIN$propAcc[t] <- length(which( estPos1$error[estPos1$inPoly==1] < accGoal ))/length(estPos1$error[estPos1$inPoly==1])
    confStatsIN$outlierCount[t] <- length(which( estPos1$error[estPos1$inPoly==1] > OutlierGoal ))
    confStatsIN$totalPoints[t] <- length(estPos1$error[estPos1$inPoly==1])
    confStatsIN$pointsP[t] <- confStatsIN$totalPoints[t]/confStats$totalPoints[t]*100
    
    # Binning by HPE
    #1. Bin by HPE
    # Create increments to bin by
    breaks <-seq(0, max(estPos1$HPE),by=1)
    # Create a column with bins
    estPos1$bin <- cut(estPos1$HPE,breaks)
    #2. Calculate the average error for each 1 unit HPE bin
    binMean <- tapply(estPos1$HPEm,estPos1$bin,mean,na.rm=TRUE)
    # count the number of HPEs that fit into each bin value
    binNum <- tapply(estPos1$HPEm,estPos1$bin,length)
    binNum[is.na(binNum)] <- 0
    # New dataframe for bin data
    bin <- data.frame(binMean, binNum)
    #3. Every HPE value in a given 1 m bin has a corresponding HPEm. Each of
    #these HPEm distances is composed of 2 elements: the error (difference
    #between the calculated and measured position) in the X direction (xe), and
    #the error in the Y direction (ye).
    estPos1$xe <- sqrt((estPos1$Easting-(estPos1East))^2)
    estPos1$ye <- sqrt((estPos1$Northing-(estPos1North))^2)
    #4. For each bin, the standard deviations of Xe and Ye are calculated.
    bin$xeSd <- tapply(estPos1$xe,estPos1$bin,sd)
    bin$yeSd <- tapply(estPos1$ye,estPos1$bin,sd)
    #5. To convert the 2-dimensional standard deviations calculated in #4 into a single measure, 
    #the 2DRMS error is calculated from the standard deviations of Xe and Ye
    bin$RMS2d <- 2*sqrt((bin$xeSd)^2 + (bin$yeSd)^2)
    #6. Now create a line plot and a dataframe just for the numbers we need, 
    #(ie when we have at least 10 tag transmissions and an HPE less than 21)
    bin$avgHPE <- tapply(estPos1$HPE,estPos1$bin,mean)
    smallBin <- bin[ which(bin$binNum > 10),]
    smallBin <- smallBin[which(smallBin$avgHPE < 21 ),]
    res3 <- lm(smallBin$RMS2d ~ smallBin$avgHPE)

    confStats$intercept[t] <- coefficients(res3)[1]
    confStats$slope[t] <- coefficients(res3)[2]
    confStats$rsquared[t] <- summary(res3)$r.squared
    confStats$nbBins[t] <- length(which(bin$binNum > 10))
    
    if (confStats$nbBins[t]>3 && confStats$rsquared[t] >0.7) {
        confStats$useful[t] <- 'Yes'}
        else {
            confStats$useful[t] <- 'No'
        }
    

    HPE<- c(2:15)# we want a list of all possible HPE cutoffs
    AllHPE <- data.frame(HPE)
    AllHPE$ptsReject <- NA # number of points removed
    AllHPE$ptsRejectP <- NA # percentage of all points
    AllHPE$ptsRetain <- NA # number of poitns retained
    AllHPE$ptsRetainP <- NA # percentage of all points
    AllHPE$incorrectReject <- NA# good= error < accGoal and unacceptably erroneous= error > accGoal
    AllHPE$incorrectRejectP <- NA #incorrectly rejected/all rejected*100
    AllHPE$incorrectRetain<- NA
    AllHPE$incorrectRetainP <- NA #incorrectly retained/all retained *100 (own def)
    AllHPE$correctReject <- NA
    AllHPE$correctRejectP <- NA #correctly rejected/all rejected *100 (own def)
    AllHPE$correctRetain <- NA
    AllHPE$correctRetainP <- NA #correctly retained/all retained *100 (own def)
    AllHPE$goodDataLossP <- NA# incorrectReject/goodpts *100
    AllHPE$badDataRetainP <- NA# incorrectRetain/badpts * 100 (own def)
    AllHPE$sdDEV<-NA
    AllHPE$avgErr<-NA
    AllHPE$maxErr <- NA
    AllHPE$countOut <- NA # amount of retained outlier points (error > outlierGoal, but HPE < filter)
    AllHPE$meanOutValue <- NA # mean of errors that are outlier errors (> outlierGoal) of retained points
    AllHPE$insideReject <- NA # amount of rejected points inside the recPoly (receiver array polygon)
    AllHPE$outsideReject <- NA # amount of rejected points outside the recPoly
    AllHPE$outsideRejectP <- NA # percentage of total rejected fish positions lying outside the recPoly
    AllHPE$spatCov <- NA # area of spatial coverage by retained fish positions

    # Number of all points that were unacceptably erroneous
    badpts <-length(estPos1$error[estPos1$error > accGoal ])
    # % of all points were unacceptably erroneous
    badptsP <-(length(estPos1$error[estPos1$error > accGoal ])/(length(estPos1$error)))*100
    #Number of all points that are good
    goodpts <-length(estPos1$error[estPos1$error <= accGoal ])
    #% of all points that are good
    goodptsP <- (length(estPos1$error[estPos1$error <= accGoal ])/(length(estPos1$error)))*100

    for( i in 1: length(AllHPE$HPE)){

        # which() gives al the indices where the logical is true => these are counted with length
    
        AllHPE$ptsReject[i] <- length(which(estPos1$HPE > AllHPE$HPE[i]))
        AllHPE$ptsRejectP[i] <- length(which(estPos1$HPE > AllHPE$HPE[i]))/(length(estPos1$error))*100 #% of all points dropped
        AllHPE$ptsRetain[i] <- length(which(estPos1$HPE < AllHPE$HPE[i]))
        AllHPE$ptsRetainP[i] <- length(which(estPos1$HPE < AllHPE$HPE[i]))/(length(estPos1$error))*100 #% of all points retained
    
        #incorrect reject: error < than acceptable error, but HPE > filter (in AllHPE)
        AllHPE$incorrectReject[i] <- length(which((estPos1$HPE[estPos1$error <=accGoal] > AllHPE$HPE[i]) == TRUE))
        AllHPE$incorrectRejectP[i] <-AllHPE$incorrectReject[i]/AllHPE$ptsReject[i]*100

        #incorrect retain
        AllHPE$incorrectRetain[i] <-length( which((estPos1$HPE[ estPos1$error >accGoal ] < AllHPE$HPE[i]) == TRUE))
        AllHPE$incorrectRetainP[i] <-AllHPE$incorrectRetain[i]/AllHPE$ptsRetain[i]*100
    
        #Correctly Rejected
        AllHPE$correctReject[i] <-(length(which((estPos1$HPE[estPos1$error > accGoal] > AllHPE$HPE[i]) == TRUE)))
        AllHPE$correctRejectP[i]<- AllHPE$correctReject[i]/AllHPE$ptsReject[i]*100
    
        #Correctly Retained
        AllHPE$correctRetain[i] <- length(which((estPos1$HPE[ estPos1$error < accGoal] < AllHPE$HPE[i]) == TRUE))
        AllHPE$correctRetainP[i]<-AllHPE$correctRetain[i]/AllHPE$ptsRetain[i]*100
    
        AllHPE$goodDataLossP[i] <- (AllHPE$incorrectReject[i] / goodpts)*100
        AllHPE$badDataRetainP[i] <- (AllHPE$incorrectRetain[i]/badpts)*100 
        AllHPE$avgErr[i] <- mean(estPos1$error[ estPos1$HPE < AllHPE$HPE[i]]) #mean of all error for each hpe
        AllHPE$sdDEV [i]<- sd(estPos1$error[ estPos1$HPE < AllHPE$HPE[i]]) #Standard deviation of all error for each HPE
        AllHPE$maxErr[i] <- max(estPos1$error[ estPos1$HPE < AllHPE$HPE[i]]) #max of all error for each hpe
        AllHPE$countOut[i] <- length(which( (estPos1$HPE[ estPos1$error > OutlierGoal] <AllHPE$HPE[i]) == TRUE )) #for each HPE nb of points with error > outliergoal
        # if you want to select the correct indexes, give criteria as AND, because subselecting in subselection => indexes have changed
        AllHPE$meanOutValue[i] <- mean(estPos1$error[which( (estPos1$HPE<AllHPE$HPE[i]&estPos1$error > OutlierGoal) == TRUE )])
    
        #Fish positions inside/outside error and spatial coverage of all fish positions
    
        AllHPE$insideReject[i] <- sum(pos$inPoly[pos$HPE>AllHPE$HPE[i]])
        AllHPE$outsideReject[i] <- length(which(pos$HPE > AllHPE$HPE[i]))-AllHPE$insideReject[i]
        AllHPE$outsideRejectP[i] <- AllHPE$outsideReject[i]/length(which(pos$HPE > AllHPE$HPE[i]))*100
        
        retained <- subset(pos, pos$HPE<AllHPE$HPE[i])
        h <- chull(retained$Easting, retained$Northing)
        AllHPE$spatCov[i] <- areapl(cbind(retained$Easting[h], retained$Northing[h]))
    
    }
    
    tests_AllHPE[[TEST]] <- AllHPE

    }

   # options(warn=-1) #turn off warnings


Warning message:
In max(estPos1$error[estPos1$HPE < AllHPE$HPE[i]]): no non-missing arguments to max; returning -InfWarning message:
In max(estPos1$error[estPos1$HPE < AllHPE$HPE[i]]): no non-missing arguments to max; returning -InfWarning message:
In max(estPos1$error[estPos1$HPE < AllHPE$HPE[i]]): no non-missing arguments to max; returning -Inf

The table below shows the calculated parameters for all ref and sync transmitters for estimating the baseline position confidence in the array.

- **posYield**: number of positions observed / max possible number of positions during the period
- **avgAcc**: mean error (goal = 0.66 m)
- **medAcc**: median error
- **propAcc**: proportion of positions that meet the accuracy goal (error < accGoal)
- **outlierCount**: number of positions that exceed the max. acceptable error (15 m)
- **totalPoints**: total number of positions calculated for that transmitter
- **nbBins**: number of bins with at least 10 tags (binWidth = 1 HPE-value)
- linear model calculated for all bins with average HPE < 21: **intercept**, **slope** and **R squared** 
- **useful**: more then 3 bins and R squared > 0.7

*PROBLEM*: the slope varies between -5 and 19 for the different stationary tags (12 if we don't count R1, which has only 2 bins). Especially negative slopes are problematic: lower errors when HPE is larger...   
It looks as if the (statistical) relationship error-HPE is very different among different stationary tags, so how to determine which of these relationships would count for fish tags?   
For S10, S12, S13, S17 and S18: Rsquared values are too low => relationship cannot be used.

In [72]:
confStats

,tests,posYield,avgAcc,medAcc,propAcc,outlierCount,totalPoints,nbBins,intercept,slope,rsquared,useful
1,R1,87.03543,1.259361,1.130591,0.9472629,19,7983,2,-30.70008,19.13045,1,No
2,R2,89.37297,1.216124,0.8478138,0.9827986,40,8197,5,-14.08632,12.31543,0.792425,Yes
3,R3,86.21215,6.448813,1.743749,0.6983685,1662,7907,7,-1.846323,6.764974,0.756447,Yes
4,S7,78.80395,20.93063,12.38225,0.343296,3327,7227,11,59.71397,-4.942258,0.5373927,No
5,S8,87.10182,0.7135164,0.697094,0.991989,10,7989,3,-0.2600634,0.7228226,0.8961728,No
6,S10,84.03773,3.346662,0.900842,0.670083,289,7708,7,21.54305,0.1571795,0.00092587,No
7,S12,82.86296,11.02394,4.091735,0.1764474,1709,7600,23,44.79086,-0.3681985,0.02216662,No
8,S13,85.05751,6.278547,2.252352,0.5292911,1130,7801,41,23.27253,-0.09547875,0.004373078,No
9,S14,86.0363,1.175656,0.9502188,0.9472817,13,7891,6,-3.045411,2.274102,0.8656384,Yes
10,S15,84.23204,11.03787,8.032306,0.3980067,2335,7726,7,6.300856,5.321627,0.5360079,No


**18 outliers within array???**

In [85]:
confStatsIN

,tests,posYield,avgAcc,medAcc,propAcc,outlierCount,totalPoints,pointsP
1,R1,0.8702452,1.255104,1.130591,0.9473816,18,7982,99.98747
2,R2,0.01035798,23.47072,7.035767,0,32,95,1.158961
3,R3,0.8619035,6.43138,1.743749,0.6985452,1660,7905,99.97471
4,S7,0.5654868,28.3769,40.10324,0.1509834,3307,5186,71.75868
5,S8,0.8710182,0.7135164,0.697094,0.991989,10,7989,100
6,S10,0.8363433,3.054761,0.8926358,0.6733151,252,7671,99.51998
7,S12,0.7933039,11.33659,4.139388,0.1562672,1701,7276,95.73684
8,S13,0.4565258,10.16891,5.00409,0.2882732,1112,4187,53.67261
9,S14,0.4119188,1.138671,0.8396086,0.9372684,10,3778,47.87733
10,S15,0.5081615,17.03121,14.98529,0.1034113,2327,4661,60.32876


pointsP: percentage of points lying within the receiver array

Below you see the plots of data loss versus retention, for the 4 stationary tags that I would consider as useful (more than 3 bins and Rsquared > 0.7).    
From high to low: R2, R3 (ref transmitters), S14, S16 (synchronization transmitters)   
Incorrectly rejecting is never a problem, but incorrectly retaining is: up to 50 % unacceptable data retention in the last case... Also the latter is a very strange plot, with some HPE-points overlaying each other...   
According to the first graph (R2), a HPE-filter of 4 would give an incorrect retention lower than 1 %, but for S16 almost half of the data would be incorrectly retained at this HPE-filter... Doesn't look promising with respect to the incorrect data that the fish tracks can contain then...

In [ ]:
display_png(file="figures/LossVSret_R2.png")
display_png(file="figures/LossVSret_R3.png")
display_png(file="figures/LossVSret_S14.png")
display_png(file="figures/LossVSret_S16.png")


The figure below shows the location of reference transmitters and receiver stations. The synchronization (sync) transmitters are colocated with the receiver stations, except for S18-2 and S9, which are pure receiver stations.   
Sync transmitters which resulted in negative slopes of the error-HPE relation, are S7, S12 and S13. These are positioned at the outside of the array, which might explain why they behave so badly.   
For the sync transmitters with very low R squared, S17 is also at the outside, but S18 is positioned quite good I would say. S10 also doesn't seem badly positioned, but being out of sight of S15 and S16 may give rise to a lot of reflected (false) positions...

In [ ]:
display_png(file="/Users/jennavergeynst/Documents/Analysis/QGis visualisaties/ReceiverArray.png")
